In [1]:
from tqdm.notebook import tqdm
from stox.alpha_vantage import AlphaVantageStocks
from stox.ibkr import load_symbol_table, save_symbol_table
import omnifig as fig
fig.initialize()

In [2]:
av = AlphaVantageStocks()
info = av.overview('BBVA')
info.keys()

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])

In [3]:
symbols_table = load_symbol_table()
eur = {yfsym: info for yfsym, info in symbols_table.items() if info['ibkr-contract']['currency'] == 'EUR'}
len(symbols_table), len(eur)

(454, 239)

In [4]:
adr_srcs = {yfsym: info['adr'] for yfsym, info in symbols_table.items() if 'adr' in info}
adrs = {yfsym: info['adr-src'] for yfsym, info in symbols_table.items() if 'adr-src' in info}
assert all(adr_srcs[adrs[yfsym]] == yfsym for yfsym in adrs)
len(adr_srcs), len(adrs)

(182, 182)

In [5]:
nys = [adr for adr in adrs if symbols_table[adr]['ibkr-contract']['primaryExchange'] in {'NYSE', 'NASDAQ'}]
len(nys)

31

In [6]:
dl = fig.create_config(_type='downloader/alpha-vantage',).pull(silent=True)
dl

| av: None (by default)
| root: None (by default)
| keys: None (by default)
| date: None (by default)


In [9]:
dl.download_reports('AAVMY', date='230909', pbar=tqdm)

  0%|          | 0/8 [00:00<?, ?it/s]

{'overview': WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/stoxdata/alpha_vantage_data/AAVMY/230909/overview.json'),
 'income_statement': WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/stoxdata/alpha_vantage_data/AAVMY/230909/income_statement.json'),
 'balance_sheet': WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/stoxdata/alpha_vantage_data/AAVMY/230909/balance_sheet.json'),
 'cash_flow': WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/stoxdata/alpha_vantage_data/AAVMY/230909/cash_flow.json'),
 'earnings': WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/stoxdata/alpha_vantage_data/AAVMY/230909/earnings.json'),
 'earnings_calendar': WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/stoxdata/alpha_vantage_data/AAVMY/230909/earnings_calendar.json'),
 'ipo_calendar': WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/stoxdata/alpha_vantage_data/AAVMY/230909/ipo_calendar.json'),
 'listing_status': WindowsPath('C:/Users/anwan/OneDrive/K